<a href="https://colab.research.google.com/github/IA2021UR/trabajo-ia-en-la-comunidad-de-la-rioja-equipo-4/blob/main/comparacion_modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Comparativa de modelos


# GAN

## Grey to RGB

### Discriminator

In [ ]:
def build_grey_discriminator():
  input_shape = (256,256, 2)
  model = Sequential()
  model.add(Conv2D(256, kernel_size=(5,5), strides=(2,2), padding="same" ,input_shape=input_shape))
  model.add(LeakyReLU())
  model.add(Dropout(0.3))
  model.add(Conv2D(128, kernel_size=(5,5), strides=(2,2), padding="same"))
  model.add(LeakyReLU())
  model.add(Dropout(0.3))
  model.add(Flatten())
  model.add(Dense(1, activation="sigmoid"))
  return model

def compile_grey_discriminator(model):
  opt = Adam(lr=.0008, beta_1=0.9)
  #model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
  model.compile(optimizer='rmsprop',loss='mse', metrics=['accuracy'])
  model.trainable = True
  return model

def save_grey_discriminator(model):
  model.save('grey_discriminator.h5', overwrite=True)

def load_grey_discriminator():
  return models.load_model('grey_discriminator.h5')

### Generator

In [ ]:
def build_grey_generator():
  g_input_shape = (256,256, 1)
  g_input = Input(shape=g_input_shape)

  conv1 = Conv2D(64, (3, 3), padding='same', strides=2)(g_input)
  conv1 = BatchNormalization()(conv1)
  conv1 = Activation('relu')(conv1)

  conv2 = Conv2D(128, (3, 3), padding='same', strides=1)(conv1)
  conv2 = BatchNormalization()(conv2)
  conv2 = Activation('relu')(conv2)

  conv3 = Conv2D(256, (3, 3), padding='same', strides=2)(conv2)
  conv3 = BatchNormalization()(conv3)
  conv3 = Activation('relu')(conv3)

  conv4 = Conv2D(512, (3, 3), padding='same', strides=1)(conv3)
  conv4 = BatchNormalization()(conv4)
  conv4 = Activation('relu')(conv4)

  conv5 = Conv2D(512, (3, 3), padding='same', strides=2)(conv4)
  conv5 = BatchNormalization()(conv5)
  conv5 = Activation('relu')(conv5)

  conv7 = UpSampling2D(size=(2, 2))(conv5)
  conv7 = Conv2D(512, (3, 3), padding='same')(conv7)
  conv7 = BatchNormalization()(conv7)
  conv7 = Activation('relu')(conv7)
  conv7 = Concatenate(axis=-1)([conv7,conv4])

  conv8 = Conv2D(512, (3, 3), padding='same')(conv7)
  conv8 = BatchNormalization()(conv8)
  conv8 = Activation('relu')(conv8)

  up2 = UpSampling2D(size=(2, 2))(conv8)
  conv9 = Conv2D(256, (3,3), padding='same')(up2)
  conv9 = BatchNormalization()(conv9)
  conv9 = Activation('relu')(conv9)
  conv9 = Concatenate(axis=-1)([conv9,conv2])

  conv10 = Conv2D(256, (3, 3), padding='same')(conv9)
  conv10 = BatchNormalization()(conv10)
  conv10 = Activation('relu')(conv10)

  up3 = UpSampling2D(size=(2, 2))(conv10)
  conv11 = Conv2D(313, (1,1), padding='same')(up3)
  conv11 = BatchNormalization()(conv11)
  conv11 = Activation('relu')(conv11)

  conv12 = Conv2D(2, (1, 1), padding='same')(conv11)
  conv12 = Activation('tanh')(conv12)

  return Model(inputs=g_input,outputs=conv12)

def compile_grey_generator(model):
  opt = Adam(lr=.0001, beta_1=0.7)
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
  return model

def save_grey_generator(model):
  model.save('grey_generator.h5', overwrite=True)

def load_grey_generator():
  return models.load_model('grey_generator.h5')



### Generator UNET

In [ ]:
def build_grey_unet():
  inputs = Input((128, 128,1))
  conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
  conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
  pool1 = MaxPool2D(pool_size=(2, 2))(conv1)

  conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
  conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
  pool2 = MaxPool2D(pool_size=(2, 2))(conv2)

  conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
  conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
  pool3 = MaxPool2D(pool_size=(2, 2))(conv3)

  conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
  conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
  drop4 = Dropout(0.5)(conv4)
  pool4 = MaxPool2D(pool_size=(2, 2))(drop4)

  conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
  conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
  drop5 = Dropout(0.5)(conv5)

  up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
  merge6 = concatenate([drop4,up6], axis = 3)
  conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
  conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

  up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
  merge7 = concatenate([conv3,up7], axis = 3)
  conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
  conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

  up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
  merge8 = concatenate([conv2,up8], axis = 3)
  conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
  conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

  up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
  merge9 = concatenate([conv1,up9], axis = 3)
  conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
  conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
  conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
  conv10 = Conv2D(2, 1, activation = 'tanh')(conv9)

  model = Model(inputs,conv10)
  return model

def compile_grey_unet(model):
  opt = Adam(lr=.002)
  #model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
  model.compile(optimizer='rmsprop',loss='mse', metrics=['accuracy'])
  return model

def save_grey_unet(model):
  model.save('grey_unet.h5', overwrite=True)

def load_grey_unet():
  return models.load_model('grey_unet.h5')


### Train

### Test

## L to LAB

# No GAN